<a href="https://colab.research.google.com/github/MKrupauskas/colab/blob/master/federated-learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install syft

     |████████████████████████████████| 256kB 2.9MB/s 
     |████████████████████████████████| 256kB 42.0MB/s 
     |████████████████████████████████| 389kB 42.6MB/s 
     |████████████████████████████████| 204kB 37.1MB/s 
     |████████████████████████████████| 2.1MB 37.1MB/s 
     |████████████████████████████████| 81kB 26.0MB/s 
     |████████████████████████████████| 460kB 43.6MB/s 
     |████████████████████████████████| 51kB 18.6MB/s 
     |████████████████████████████████| 266kB 35.8MB/s 
     |████████████████████████████████| 122kB 41.4MB/s 
  Created wheel for zstd: filename=zstd-1.4.1.0-cp36-cp36m-linux_x86_64.whl size=1067107 sha256=9c296b249694a056b318906c37b94ddeac6f9e71d29e7d0dc70346f62f45cd8a
  Stored in directory: /root/.cache/pip/wheels/66/3f/ee/ac08c81af7c1b24a80c746df669ea3cb37542d27877d66ccf4
  Created wheel for pyyaml: filename=PyYAML-5.1.2-cp36-cp36m-linux_x86_64.whl size=44105 sha256=ab077a48383a1f1265f05e14554a539b234a549804962c7b3f2e2b08064e6661
  Stored in di

In [4]:
import torch as th
import syft as sy

hook = sy.TorchHook(th)

W0817 10:42:53.429933 139693912590208 secure_random.py:26] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/usr/local/lib/python3.6/dist-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.14.0.so'
W0817 10:42:53.451987 139693912590208 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tf_encrypted/session.py:26: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



In [0]:
bob = sy.VirtualWorker(hook, id = "bob")

In [6]:
bob._objects

{}

In [0]:
x = th.tensor([1, 2, 3, 4, 5])

x = x.send(bob)

In [8]:
bob._objects

{97376672822: tensor([1, 2, 3, 4, 5])}

In [9]:
alice = sy.VirtualWorker(hook, id = "alice")

x = th.tensor([1])

x_pointer = x.send(bob, alice)

x_pointer.get(sum_results = True)

tensor([2])

Remote arithmetic

In [0]:
x = th.tensor([1, 2, 3, 4, 5]).send(bob)
y = th.tensor([1, 1, 1, 1, 1]).send(bob)

In [0]:
z = x + y

z = z.get()

In [12]:
z

tensor([2, 3, 4, 5, 6])

Linear model on bob's machine

In [0]:
input = th.tensor([[1., 1], [0, 1,], [1, 0], [0, 0]], requires_grad = True).send(bob)
target = th.tensor([[1.], [1], [0], [0]], requires_grad = True).send(bob)

In [0]:
weights = th.tensor([[0.], [0.]], requires_grad = True).send(bob)

In [25]:
for i in range(10):
  pred = input.mm(weights)

  loss = ((pred - target)**2).sum()

  loss.backward()

  weights.data.sub_(weights.grad * 0.1)
  weights.grad *= 0

  print(loss.get().data)

tensor(1.5509)
tensor(0.5217)
tensor(0.2409)
tensor(0.1393)
tensor(0.0868)
tensor(0.0552)
tensor(0.0352)
tensor(0.0225)
tensor(0.0144)
tensor(0.0092)


Garbage collection is enabled by default with PySyft

Federated learning

In [0]:
from torch import nn, optim

In [0]:
data = th.tensor([[1., 1], [0, 1,], [1, 0], [0, 0]], requires_grad = True)
target = th.tensor([[1.], [1], [0], [0]], requires_grad = True)

In [0]:
model = nn.Linear(2, 1)

In [0]:
optimizer = optim.SGD(params = model.parameters(), lr = 0.1)

In [0]:
def train(iterations = 20):
  for i in range(iterations):
    optimizer.zero_grad()

    prediction = model(data)

    loss = ((prediction - target) ** 2).sum()

    loss.backward()

    optimizer.step()

    print(loss)

In [0]:
data_bob = data[0 : 2].send(bob)
target_bob = target[0 : 2].send(bob)

data_alice = data[0 : 2].send(alice)
target_alice = target[0 : 2].send(alice)

datasets = [(data_bob, target_bob), (data_alice, target_alice)]

In [0]:
model = nn.Linear(2, 1)
optimizer = optim.SGD(params = model.parameters(), lr = 0.1)

In [0]:
def remote_train(iterations = 20):
  for i in range(iterations):
    for _data, _target in datasets:
      model = model.send(_data.location)

      optimizer.zero_grad()

      prediction = model(_data)

      loss = ((prediction - _target) ** 2).sum()

      loss.backward()

      optimizer.step()

      model = model.get()

      print(loss)

In [62]:
bob.clear_objects()
alice.clear_objects()

<VirtualWorker id:alice #objects:0>

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [0]:
x = x.send(alice)

In [0]:
# remote_get is like get on remote workers (move is similar)